In [ ]:
import mujoco
import mediapy as media
import mujoco.viewer as viewer
import pathlib
import matplotlib.pyplot as plt

In [ ]:
model = mujoco.MjModel.from_xml_path('../build/mjpc/tasks/h1/walk/task.xml')
data = mujoco.MjData(model)
print("Model info:")
print(" nq", model.nq)
print(" nv", model.nv)
print(" nu", model.nu)
print(" na", model.na)
print(" njoint", model.njnt)

print("\nModel joints:")
for i in range(model.njnt):
    print("",model.joint(i).name, model.joint(i).type, model.joint(i).qposadr, model.joint(i).dofadr, model.joint(i).qpos0, model.joint(i).range)
    
print("\nModel bodies:")
for i in range(model.nbody):
    print("",model.body(i).name, i, model.body(i).pos)
    
print("\nModel sensors:")
for i in range(model.nsensor):
    print("",model.sensor(i).name, i, model.sensor(i).type, data.sensor(i).data)
    

In [ ]:
data.body('right_ankle_link')

In [ ]:
model.opt.timestep = 0.03

In [ ]:
geom = mujoco.MjvGeom()
start_pos = [0, 0, 0]
end_pos = [1, 1, 1]
width = 0.1
type = mujoco.mjtGeom.mjGEOM_LINE


In [ ]:
def add_fw_cost(manager):
    line = manager.add_line()
    line.start = lambda: data.site('upper_torso').xpos
    line.end = lambda: data.site('upper_torso').xpos + data.sensor('torso_forward').data
    line = manager.add_line()
    line.start = lambda: data.body('pelvis').xpos
    line.end = lambda: data.body('pelvis').xpos + data.sensor('pelvis_forward').data
    line = manager.add_line()
    line.start = lambda: data.body('right_ankle_link').xpos
    line.end = lambda: data.body('right_ankle_link').xpos + data.sensor('foot_right_forward').data
    line = manager.add_line()
    line.start = lambda: data.body('left_ankle_link').xpos
    line.end = lambda: data.body('left_ankle_link').xpos + data.sensor('foot_left_forward').data

In [ ]:
import time
import mujoco
import mujoco.viewer
from mjv_utils import Line, GeomManager

with mujoco.viewer.launch_passive(model, data) as viewer:
  start = time.time()
  while not viewer.is_running():
    pass
  viewer.opt.flags[mujoco.mjtVisFlag.mjVIS_TRANSPARENT] = 1
  manager = GeomManager(viewer.user_scn)
  add_fw_cost(manager)
  while viewer.is_running():
    step_start = time.time()

    # mj_step can be replaced with code that also evaluates
    # a policy and applies a control signal before stepping the physics.
    mujoco.mj_step(model, data)
    manager.update()
    
    # Example modification of a viewer option: toggle contact points every two seconds.
    # with viewer.lock():
    #   viewer.opt.flags[mujoco.mjtVisFlag.mjVIS_CONTACTPOINT] = int(data.time % 2)
    
    # Pick up changes to the physics state, apply perturbations, update options from GUI.
    viewer.sync()

    # Rudimentary time keeping, will drift relative to wall clock.
    time_until_next_step = model.opt.timestep - (time.time() - step_start)
    
    if time_until_next_step > 0:
      time.sleep(time_until_next_step)
    else:
      print("Simulation not running in real time!")

In [ ]:
model = mujoco.MjModel.from_xml_path('./build/mjpc/tasks/humanoid/walk/task.xml')
data = mujoco.MjData(model)
viewer.launch(model=model, data=data)

In [ ]:
model = mujoco.MjModel.from_xml_path('./mjpc/tasks/h1/walk/task.xml')
data = mujoco.MjData(model)
viewer.launch(model=model, data=data)

# Trajectory interpolation testing

In [ ]:
import pathlib
import mujoco
from mujoco_mpc import agent as agent_lib
import time
import scipy
import scipy.interpolate

In [ ]:

model = mujoco.MjModel.from_xml_path('../build/mjpc/tasks/h1/walk/task.xml')
data = mujoco.MjData(model)

agent = agent_lib.Agent(task_id="H1 Walk", 
                            model=model, 
                            server_binary_path=pathlib.Path(agent_lib.__file__).parent
                            / "mjpc"
                            / "agent_server")

agent2 = agent_lib.Agent(task_id="H1 Walk", 
                            model=model, 
                            server_binary_path=pathlib.Path(agent_lib.__file__).parent
                            / "mjpc"
                            / "agent_server")

agent.set_state(
                time=data.time+1,
                qpos=data.qpos,
                qvel=data.qvel,
                act=data.act,
                mocap_pos=data.mocap_pos,
                mocap_quat=data.mocap_quat,
                userdata=data.userdata,
            )

agent2.set_state(
                time=data.time,
                qpos=data.qpos,
                qvel=data.qvel,
                act=data.act,
                mocap_pos=data.mocap_pos,
                mocap_quat=data.mocap_quat,
                userdata=data.userdata,
            )

In [ ]:
agent.get_cost_weights()

In [ ]:
t = time.time()
agent.planner_step()
agent2.planner_step()
print("Time", time.time()-t)

In [ ]:
t = time.time()
best_traj = agent.best_trajectory()
elapsed = time.time() - t
print(elapsed)
best_traj

In [ ]:
t = time.time()
agent.get_action(nominal_action=True, time=0.0000)
elapsed = time.time() - t
print(elapsed)

In [ ]:
print(len(best_traj['times']), len(best_traj['actions']), len(best_traj['states']))

In [ ]:
t = time.time()
a = scipy.interpolate.interp1d(best_traj['times'][:-1], best_traj['actions'], axis=0, kind='linear', fill_value='extrapolate')
elapsed = time.time() - t
print(elapsed)
t = time.time()
a(0.01)
elapsed = time.time() - t
print(elapsed)

In [ ]:
model_path = (
        "../build/mjpc/tasks/panda/task.xml"#"../build/mjpc/tasks/h1/walk/task.xml"
    )
model = mujoco.MjModel.from_xml_path(str(model_path))
data = mujoco.MjData(model)
mujoco.mj_resetData(model, data)
mujoco.mj_step(model, data)
width, height = (640, 480)
renderer = mujoco.Renderer(model)
renderer.update_scene(data)
fig = renderer.render()
plt.imshow(fig)
